In [1]:
import os
import pandas as pd

# ----------------------------------
# 0. 파일 경로 설정
# ----------------------------------
cohort_path = "./cohort/cohort_ver42_with_troponin.csv"
ecg_path = "../../data/mimic-iv-ecg/machine_measurements.csv"
output_path = "./cohort/cohort_ver43_with_ecg.csv"

# ----------------------------------
# 1. Cohort 불러오기
# ----------------------------------
if not os.path.exists(cohort_path):
    raise FileNotFoundError(f"cohort 파일을 찾을 수 없습니다: {cohort_path}")

cohort = pd.read_csv(cohort_path)
cohort["hadm_id"] = cohort["hadm_id"].astype("Int64")

print("[INFO] cohort 로드 완료:", cohort.shape)

# ----------------------------------
# 2. ECG 데이터 로드
# ----------------------------------
if not os.path.exists(ecg_path):
    raise FileNotFoundError(f"ECG 파일을 찾을 수 없습니다: {ecg_path}")

ecg = pd.read_csv(ecg_path, low_memory=False)
print("[INFO] raw ECG shape:", ecg.shape)

# ----------------------------------
# 2-1. 시간 컬럼 선택 (charttime 또는 ecg_time)
# ----------------------------------
if "charttime" in ecg.columns:
    time_col = "charttime"
elif "ecg_time" in ecg.columns:
    time_col = "ecg_time"
else:
    raise ValueError("ECG 파일에 charttime 또는 ecg_time 컬럼이 없습니다.")

ecg[time_col] = pd.to_datetime(ecg[time_col], errors="coerce")

# ----------------------------------
# 2-2. ID 기준 (hadm_id 또는 subject_id)
# ----------------------------------
if "hadm_id" in ecg.columns:
    id_col = "hadm_id"
    ecg[id_col] = ecg[id_col].astype("Int64")
else:
    # machine_measurements.csv 구조에는 hadm_id가 없음 → subject_id 사용
    id_col = "subject_id"
    ecg[id_col] = ecg[id_col].astype("Int64")
    print("[WARN] ECG에 hadm_id가 없습니다. subject_id 기준으로 집계합니다.")

# ----------------------------------
# 2-3. STEMI 텍스트 컬럼 구성
# ----------------------------------
if "machine_measurements" in ecg.columns:
    text_col = "machine_measurements"
    ecg[text_col] = ecg[text_col].astype(str)
else:
    # report_0~report_17 합치기
    report_cols = [c for c in ecg.columns if c.startswith("report_")]
    if len(report_cols) == 0:
        raise ValueError("report_0~report_17 컬럼이 없습니다. STEMI 판정이 불가합니다.")

    text_col = "full_report"
    ecg[text_col] = (
        ecg[report_cols]
        .fillna("")
        .astype(str)
        .agg(" ".join, axis=1)
    )

print(f"[INFO] ECG text column = {text_col}")

# NaN ID 제거
ecg = ecg[ecg[id_col].notna() & ecg[time_col].notna()].copy()
print("[INFO] 유효 ECG 행 수:", len(ecg))

# ----------------------------------
# 2-4. STEMI 판정 함수
# ----------------------------------
def detect_stemi(txt: str) -> bool:
    if not isinstance(txt, str):
        return False
    t = txt.lower()
    
    patterns = [
        "acute st elevation mi",
        "consider acute st elevation mi",
        "st elevation, consider acute infarct",
        "stemi",  # 기본 STEMI 키워드
    ]
    return any(p in t for p in patterns)

ecg["is_stemi"] = ecg[text_col].apply(detect_stemi)

print("[INFO] STEMI ECG 개수:", ecg["is_stemi"].sum())

# ----------------------------------
# 3. ECG 4개 파생변수 계산
# ----------------------------------

# 1) first_ecg_charttime
first_ecg = (
    ecg.groupby(id_col)[time_col]
    .min()
    .reset_index()
    .rename(columns={time_col: "first_ecg_charttime"})
)

# 2) ecg_count
ecg_count = (
    ecg.groupby(id_col)[time_col]
    .count()
    .reset_index()
    .rename(columns={time_col: "ecg_count"})
)

# STEMI ECG만
ecg_stemi = ecg[ecg["is_stemi"]].copy()

# 3) first_stemi_ecg_time
first_stemi_ecg = (
    ecg_stemi.groupby(id_col)[time_col]
    .min()
    .reset_index()
    .rename(columns={time_col: "first_stemi_ecg_time"})
)

# 4) ecg_stemi_count
ecg_stemi_count = (
    ecg_stemi.groupby(id_col)[time_col]
    .count()
    .reset_index()
    .rename(columns={time_col: "ecg_stemi_count"})
)

# ----------------------------------
# 4. Cohort에 4개 컬럼 merge
# ----------------------------------
if id_col not in cohort.columns:
    raise ValueError(f"cohort에 '{id_col}'가 없어 ECG 데이터를 merge할 수 없습니다.")

cohort = cohort.merge(first_ecg, on=id_col, how="left")
cohort = cohort.merge(ecg_count, on=id_col, how="left")
cohort = cohort.merge(first_stemi_ecg, on=id_col, how="left")
cohort = cohort.merge(ecg_stemi_count, on=id_col, how="left")

# 결측 0처리
cohort["ecg_count"] = cohort["ecg_count"].fillna(0).astype(int)
cohort["ecg_stemi_count"] = cohort["ecg_stemi_count"].fillna(0).astype(int)

print("[INFO] ECG 4개 컬럼 merge 완료")

# ----------------------------------
# 5. 저장
# ----------------------------------
os.makedirs(os.path.dirname(output_path), exist_ok=True)
cohort.to_csv(output_path, index=False)

print(f"[DONE] ECG 4개 컬럼 추가 완료 → {output_path}")


[INFO] cohort 로드 완료: (1930, 18)
[INFO] raw ECG shape: (800035, 33)
[WARN] ECG에 hadm_id가 없습니다. subject_id 기준으로 집계합니다.
[INFO] ECG text column = full_report
[INFO] 유효 ECG 행 수: 800035
[INFO] STEMI ECG 개수: 9079
[INFO] ECG 4개 컬럼 merge 완료
[DONE] ECG 4개 컬럼 추가 완료 → ./cohort/cohort_ver43_with_ecg.csv
